In [1]:
import numpy as np
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(".."))
from scripts.system_efficiency import calibrate_system_efficiency

In [2]:
# # Simulate a dummy input dataframe
np.random.seed(0)
# Create an 8-day hourly time series
hours_per_day = 24
total_days = 8
timestamps = pd.date_range("2022-06-01", periods=total_days * hours_per_day, freq="h", tz="UTC")

df = pd.DataFrame({'timestamp': timestamps})
df['date'] = df['timestamp'].dt.date

# Simulate modeled clear-sky power
df['Pcs'] = 500 * np.clip(np.sin(np.linspace(0, 3 * np.pi, len(df))), 0, 1)

# Default values: clear and cool
df['Pm'] = df['Pcs'] + np.random.normal(0, 15, len(df))
df['temperature'] = 20
df['cloudiness'] = 1

# Weather features
df['dew_point'] = df['temperature'] - 5 + np.random.normal(0, 1, len(df))
df['humidity'] = 55 + 10 * np.sin(np.linspace(0, 3 * np.pi, len(df))) + np.random.randn(len(df))
df['wind_speed'] = 2 + np.abs(np.random.normal(0, 1, len(df)))
df['wind_direction'] = np.random.randint(0, 360, len(df))
df['mean_sea_level_pressure'] = 1013 + np.random.normal(0, 3, len(df))

# June 1–3: Clear + cool → trigger kappa_c
cool_mask = df['date'] <= pd.to_datetime("2022-06-03").date()
df.loc[cool_mask, 'temperature'] = 20
df.loc[cool_mask, 'cloudiness'] = 1
df.loc[cool_mask, 'Pm'] = df.loc[cool_mask, 'Pcs'] * 0.92 + np.random.normal(0, 10, cool_mask.sum())

# June 4–6: Clear + warm → trigger kappa_t
warm_mask = df['date'].between(pd.to_datetime("2022-06-04").date(), pd.to_datetime("2022-06-06").date())
df.loc[warm_mask, 'temperature'] = 28
df.loc[warm_mask, 'cloudiness'] = 1
df.loc[warm_mask, 'Pm'] = df.loc[warm_mask, 'Pcs'] * 0.85 + np.random.normal(0, 10, warm_mask.sum())

# June 7–8: Not clear → test non-calibration fallback
cloudy_mask = df['date'] >= pd.to_datetime("2022-06-07").date()
df.loc[cloudy_mask, 'cloudiness'] = 7
df.loc[cloudy_mask, 'Pm'] = df.loc[cloudy_mask, 'Pcs'] * 0.5 + np.random.normal(0, 20, cloudy_mask.sum())

# Clean up
df = df.drop(columns='date')

In [3]:
df.head()

,timestamp,Pcs,Pm,temperature,cloudiness,dew_point,humidity,wind_speed,wind_direction,mean_sea_level_pressure
0,2022-06-01 00:00:00+00:00,0.000000,-3.007839,20,1,14.960717,55.038631,2.805627,202,1012.655674
1,2022-06-01 01:00:00+00:00,24.662183,16.083122,20,1,13.831907,53.836529,3.118312,133,1013.926254
2,2022-06-01 02:00:00+00:00,49.264328,47.421677,20,1,15.523277,54.999776,2.131054,340,1008.887720
3,2022-06-01 03:00:00+00:00,73.746546,55.440576,20,1,14.828454,55.003096,3.133080,178,1015.596959
4,2022-06-01 04:00:00+00:00,98.049237,92.429930,20,1,15.771791,58.609120,3.951804,86,1016.244128


In [5]:
df, (kc, kt) = calibrate_system_efficiency(df,
    pm_col="Pm",
    pcs_col="Pcs",
    cloud_col="cloudiness",
    temp_col="temperature")

[INFO] Calibrated κc = 0.9193 (cool window)


In [7]:
print(f"Final κc: {kc:.4f}, κt: {kt:.4f}")


Final κc: 0.9193, κt: 1.0000


In [8]:
df.head()

,timestamp,Pcs,Pm,temperature,cloudiness,dew_point,humidity,wind_speed,wind_direction,mean_sea_level_pressure,Pccs_kW
0,2022-06-01 00:00:00+00:00,0.000000,-3.007839,20,1,14.960717,55.038631,2.805627,202,1012.655674,0.000000
1,2022-06-01 01:00:00+00:00,24.662183,16.083122,20,1,13.831907,53.836529,3.118312,133,1013.926254,22.671237
2,2022-06-01 02:00:00+00:00,49.264328,47.421677,20,1,15.523277,54.999776,2.131054,340,1008.887720,45.287284
3,2022-06-01 03:00:00+00:00,73.746546,55.440576,20,1,14.828454,55.003096,3.133080,178,1015.596959,67.793085
4,2022-06-01 04:00:00+00:00,98.049237,92.429930,20,1,15.771791,58.609120,3.951804,86,1016.244128,90.133852


In [9]:
# from partial_shadding_detection import detect_power_drop_events, find_common_shading_angle, estimate_kappa_ps, apply_kappa_correction
# from partial_shadding_detection import partial_shadding_detection_pipeline

In [10]:
def add_synthetic_zenith(df: pd.DataFrame, timezone_offset: int = 0) -> pd.DataFrame:
    """
    Adds a synthetic zenith angle column based on time of day.

    Parameters:
        df (pd.DataFrame): DataFrame with 'timestamp' column.
        timezone_offset (int): Local timezone offset from UTC in hours.

    Returns:
        pd.DataFrame: DataFrame with new 'zenith' column.
    """
    df = df.copy()
    
    # Calculate local solar hour
    local_hour = (df['timestamp'].dt.hour + timezone_offset) % 24
    hours_from_noon = np.abs(local_hour - 12)

    # Simulate zenith: max at midnight (90°), min at noon (10°)
    df['zenith'] = 90 - 80 * np.cos((hours_from_noon / 12) * np.pi)

    return df

In [11]:
df = add_synthetic_zenith(df)

In [12]:
df.head()

,timestamp,Pcs,Pm,temperature,cloudiness,dew_point,humidity,wind_speed,wind_direction,mean_sea_level_pressure,Pccs_kW,zenith
0,2022-06-01 00:00:00+00:00,0.000000,-3.007839,20,1,14.960717,55.038631,2.805627,202,1012.655674,0.000000,170.000000
1,2022-06-01 01:00:00+00:00,24.662183,16.083122,20,1,13.831907,53.836529,3.118312,133,1013.926254,22.671237,167.274066
2,2022-06-01 02:00:00+00:00,49.264328,47.421677,20,1,15.523277,54.999776,2.131054,340,1008.887720,45.287284,159.282032
3,2022-06-01 03:00:00+00:00,73.746546,55.440576,20,1,14.828454,55.003096,3.133080,178,1015.596959,67.793085,146.568542
4,2022-06-01 04:00:00+00:00,98.049237,92.429930,20,1,15.771791,58.609120,3.951804,86,1016.244128,90.133852,130.000000


In [15]:
from scripts.partial_shadding_detection import detect_power_drop_events, find_common_shading_angle, estimate_kappa_ps, apply_kappa_correction
from scripts.partial_shadding_detection import partial_shading_detection

In [ ]:
df = partial_shading_detection(df, zenith_col="zenith", power_col="Pm", pccs_col="Pccs")

In [13]:
df

,timestamp,Pcs,Pm,temperature,cloudiness,dew_point,humidity,wind_speed,wind_direction,mean_sea_level_pressure,Pccs,zenith,Ppccs
0,2022-06-01 00:00:00+00:00,0.000000e+00,-3.007839,20,1,14.960717,55.038631,2.805627,202,1012.655674,0.000000e+00,170.000000,0.000000e+00
1,2022-06-01 01:00:00+00:00,2.466218e+01,16.083122,20,1,13.831907,53.836529,3.118312,133,1013.926254,2.267124e+01,167.274066,2.267124e+01
2,2022-06-01 02:00:00+00:00,4.926433e+01,47.421677,20,1,15.523277,54.999776,2.131054,340,1008.887720,4.528728e+01,159.282032,4.528728e+01
3,2022-06-01 03:00:00+00:00,7.374655e+01,55.440576,20,1,14.828454,55.003096,3.133080,178,1015.596959,6.779308e+01,146.568542,6.779308e+01
4,2022-06-01 04:00:00+00:00,9.804924e+01,92.429930,20,1,15.771791,58.609120,3.951804,86,1016.244128,9.013385e+01,130.000000,9.013385e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2022-06-08 19:00:00+00:00,9.804924e+01,59.826508,20,7,15.880179,55.131244,2.024612,19,1016.147280,9.013385e+01,110.705524,9.013385e+01
188,2022-06-08 20:00:00+00:00,7.374655e+01,51.521753,20,7,13.301894,56.511937,2.719984,197,1022.512924,6.779308e+01,130.000000,6.779308e+01
189,2022-06-08 21:00:00+00:00,4.926433e+01,17.127716,20,7,15.387280,56.753189,3.102906,212,1013.568499,4.528728e+01,146.568542,4.528728e+01
190,2022-06-08 22:00:00+00:00,2.466218e+01,6.498252,20,7,12.744436,56.083123,2.101697,125,1008.954761,2.267124e+01,159.282032,2.267124e+01
